# Iterative Testing

In [ ]:
import time
import numpy as np
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
import glob
import os

# Ensure plots display inline in Jupyter Notebook
%matplotlib inline

device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_inputs_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/train_inputs_norm.pt').to(device)
train_targets_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/train_targets_norm.pt').to(device)
print(f"Tensors loaded: train inputs {train_inputs_tensor.shape}, train targets {train_targets_tensor.shape}")
print(f"Normalized t2m_f mean: {train_targets_tensor[:, :, 5, :].mean().item():.2f}, std: {train_targets_tensor[:, :, 5, :].std().item():.2f}")

# Temporary Train/Test split
train_size = int(0.9 * train_inputs_tensor.shape[0])
print(f"Train size: {train_size}")
temp_train_inputs = train_inputs_tensor[:train_size]
temp_train_targets = train_targets_tensor[:train_size]
temp_test_inputs = train_inputs_tensor[train_size:]
temp_test_targets = train_targets_tensor[train_size:]
print(f"Train/test split: train shape {temp_train_inputs.shape}, test shape {temp_test_inputs.shape}")

# Move tensors to GPU upfront
temp_train_inputs = temp_train_inputs.to(device)
temp_train_targets = temp_train_targets.to(device)
temp_test_inputs = temp_test_inputs.to(device)
temp_test_targets = temp_test_targets.to(device)

# Define grid and edge index (must match training setup)
num_nodes = 23937
k = 8
lat_subset = np.linspace(50, 25, 101)
lon_subset = np.linspace(235, 294, 237)
coords = np.stack(np.meshgrid(lat_subset, lon_subset, indexing='ij'), axis=-1).reshape(-1, 2)
nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)
_, indices = nbrs.kneighbors(coords)
edge_index = t.tensor(np.stack([np.repeat(np.arange(num_nodes), k), indices[:, 1:].flatten()]), dtype=t.long).to(device)

# Model definition (must match the trained model)
class WeatherGNN(t.nn.Module):
    def __init__(self, num_features=15, hidden_dims=128, num_outputs=1):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dims)
        self.conv2 = GCNConv(hidden_dims, hidden_dims)
        self.conv3 = GCNConv(hidden_dims, num_outputs)
        self.dropout = t.nn.Dropout(0.3)
        self.residual = t.nn.Linear(num_features, num_outputs)
        self.res_weight = t.nn.Parameter(t.tensor(2.0))

    def forward(self, x, edge_index):
        residual = self.residual(x) * self.res_weight
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x + residual

# Define L1 loss function directly
def l1_loss(x, y):
    return t.mean(t.abs(x - y))

# Pre-compute t2m_f statistics for denormalization (must match training)
t2m_f_mean = 42.36  # °F
t2m_f_std = 21.75   # °F

# Testing function with iterative predictions
def test_model_iterative(model_path, inputs_tensor, targets_tensor, edge_index, num_nodes, device, t2m_f_mean, t2m_f_std, lat_subset, lon_subset):
    model = WeatherGNN(num_features=15, hidden_dims=128, num_outputs=1).to(device)
    model.load_state_dict(t.load(model_path))
    model.eval()
    print(f"Model loaded from {model_path}")

    # Validate inputs
    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Targets tensor shape: {targets_tensor.shape}")
    if inputs_tensor.shape[0] < 2 or targets_tensor.shape[0] < 2:
        raise ValueError("Input and target tensors must have at least 2 time steps for prediction.")

    # Iterative prediction
    test_preds = []
    current_input = inputs_tensor[0].clone()  # Shape: [num_nodes, num_features, 1]
    t2m_f_idx = 0  # Index of t2m_f in the feature dimension (after dropping t2m, d2m, d2m_f, t)

    for t_step in range(inputs_tensor.shape[0] - 1):
        # Reshape input for the model
        input_x = current_input.reshape(num_nodes, -1)  # Shape: [num_nodes, num_features]
        
        # Predict t2m_f for the next time step
        with t.no_grad():
            out = model(input_x, edge_index)  # Shape: [num_nodes, 1]
        test_preds.append(out)

        # Prepare input for the next time step
        if t_step < inputs_tensor.shape[0] - 2:
            # Use true inputs for all features except t2m_f
            current_input = inputs_tensor[t_step + 1].clone()  # Shape: [num_nodes, num_features, 1]
            # Replace the t2m_f feature with the normalized prediction
            normalized_pred = (out - t2m_f_mean) / t2m_f_std  # Shape: [num_nodes, 1]
            current_input[:, t2m_f_idx, 0] = normalized_pred.squeeze(-1)  # Shape: [num_nodes]

    test_preds = t.stack(test_preds)  # Shape: [num_time_steps-1, num_nodes, 1], on GPU
    test_trues = targets_tensor[:-1, :, 5, :].reshape(targets_tensor.shape[0]-1, num_nodes, 1)  # Shape: [num_time_steps-1, num_nodes, 1], on GPU

    # Denormalize for evaluation and plotting
    preds_t2m = test_preds * t2m_f_std + t2m_f_mean  # On GPU
    trues_t2m = test_trues * t2m_f_std + t2m_f_mean  # On GPU
    mae_t2m = t.mean(t.abs(preds_t2m - trues_t2m)).item()
    rmse_t2m = t.sqrt(t.mean((preds_t2m - trues_t2m) ** 2)).item()
    print(f"Iterative t2m_f L1 norm (°F): {mae_t2m:.2f}")
    print(f"Iterative t2m_f RMSE (°F): {rmse_t2m:.2f}")

    # Move to CPU for plotting
    preds_t2m = preds_t2m.cpu()
    trues_t2m = trues_t2m.cpu()

    # Plot variation at specific nodes in subplots
    nodes_to_plot = [0, 236, 8926, 23500]  # Example: NW, NE, Central, SW
    node_labels = ['NW (Node 0)', 'NE (Node 236)', 'Central (Node 8926)', 'SW (Node 23500)']
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    for idx, node, label in zip(range(len(nodes_to_plot)), nodes_to_plot, node_labels):
        ax = axes[idx]
        ax.plot(preds_t2m[:, node, 0].numpy(), label='Iterative Pred', linestyle='--', color='red')
        ax.plot(trues_t2m[:, node, 0].numpy(), label='True', linestyle='-', color='green')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Temperature (°F)')
        ax.set_title(f'Iterative Predicted vs True t2m_f at {label}')
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    plt.savefig('t2m_f_iterative_variation_nodes_subplots.png')
    plt.show()

    # Debug and plot spatial difference for first time step with lat/lon grid overlay
    first_pred = preds_t2m[0, :, 0].numpy()
    first_true = trues_t2m[0, :, 0].numpy()
    print(f"First predicted t2m_f shape: {first_pred.shape}")
    print(f"First true t2m_f shape: {first_true.shape}")
    print(f"First predicted t2m_f sample: {first_pred[:5]}")
    print(f"First true t2m_f sample: {first_true[:5]}")

    # Check for NaN or infinite values
    if np.any(np.isnan(first_pred)) or np.any(np.isnan(first_true)):
        raise ValueError("NaN values detected in predictions or true values.")
    if np.any(np.isinf(first_pred)) or np.any(np.isinf(first_true)):
        raise ValueError("Infinite values detected in predictions or true values.")

    diff = np.abs(first_pred - first_true)
    print(f"Difference min: {np.min(diff):.2f}, max: {np.max(diff):.2f}, mean: {np.mean(diff):.2f}")

    # Reshape difference to 2D grid
    if diff.shape[0] != num_nodes:
        raise ValueError(f"Expected {num_nodes} nodes, but got {diff.shape[0]}")
    if len(lat_subset) * len(lon_subset) != num_nodes:
        raise ValueError(f"Grid size mismatch: lat_subset ({len(lat_subset)}) * lon_subset ({len(lon_subset)}) != num_nodes ({num_nodes})")
    diff_grid = diff.reshape(len(lat_subset), len(lon_subset))

    plt.figure(figsize=(15, 5))
    contour = plt.contourf(lon_subset, lat_subset, diff_grid, cmap='RdBu_r', levels=np.linspace(0, max(np.max(diff), 1), 21), alpha=0.8)
    plt.colorbar(label='Absolute Difference (°F)')
    
    # Overlay latitude and longitude grid lines
    plt.grid(True, linestyle='--', color='gray', alpha=0.3)  # Subtle grid lines
    plt.xticks(ticks=np.linspace(min(lon_subset), max(lon_subset), 10), rotation=45)
    plt.yticks(ticks=np.linspace(min(lat_subset), max(lat_subset), 10))
    
    plt.title('Iterative Absolute Difference (Pred - True) for t2m_f at First Test Time Step')
    plt.xlabel('Longitude (°E)')
    plt.ylabel('Latitude (°N)')
    save_path = 't2m_f_iterative_grid_difference.png'
    plt.savefig(save_path)
    print(f"Spatial grid plot saved to: {os.path.abspath(save_path)}")
    plt.show()

# Run the test with iterative predictions
test_model_iterative(
    model_path=r'F:\weather_forecasting\notebooks\final project\models\Final Model\final_model_ext_training.pth',
    inputs_tensor=temp_test_inputs,
    targets_tensor=temp_test_targets,
    edge_index=edge_index,
    num_nodes=num_nodes,
    device=device,
    t2m_f_mean=t2m_f_mean,
    t2m_f_std=t2m_f_std,
    lat_subset=lat_subset,
    lon_subset=lon_subset
)

# Specifying Testing at Major Cities

In [ ]:
import time
import numpy as np
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
import glob
import os

# Ensure plots display inline in Jupyter Notebook
%matplotlib inline

device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_inputs_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/train_inputs_norm.pt').to(device)
train_targets_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/train_targets_norm.pt').to(device)
print(f"Tensors loaded: train inputs {train_inputs_tensor.shape}, train targets {train_targets_tensor.shape}")
print(f"Normalized t2m_f mean: {train_targets_tensor[:, :, 5, :].mean().item():.2f}, std: {train_targets_tensor[:, :, 5, :].std().item():.2f}")

# Temporary Train/Test split
train_size = int(0.9 * train_inputs_tensor.shape[0])
print(f"Train size: {train_size}")
temp_train_inputs = train_inputs_tensor[:train_size]
temp_train_targets = train_targets_tensor[:train_size]
temp_test_inputs = train_inputs_tensor[train_size:]
temp_test_targets = train_targets_tensor[train_size:]
print(f"Train/test split: train shape {temp_train_inputs.shape}, test shape {temp_test_inputs.shape}")

# Move tensors to GPU upfront
temp_train_inputs = temp_train_inputs.to(device)
temp_train_targets = temp_train_targets.to(device)
temp_test_inputs = temp_test_inputs.to(device)
temp_test_targets = temp_test_targets.to(device)

# Define grid and edge index (must match training setup)
num_nodes = 23937
k = 8
lat_subset = np.linspace(50, 25, 101)
lon_subset = np.linspace(235, 294, 237)
coords = np.stack(np.meshgrid(lat_subset, lon_subset, indexing='ij'), axis=-1).reshape(-1, 2)
nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)
_, indices = nbrs.kneighbors(coords)
edge_index = t.tensor(np.stack([np.repeat(np.arange(num_nodes), k), indices[:, 1:].flatten()]), dtype=t.long).to(device)

# Model definition (must match the trained model)
class WeatherGNN(t.nn.Module):
    def __init__(self, num_features=15, hidden_dims=128, num_outputs=1):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dims)
        self.conv2 = GCNConv(hidden_dims, hidden_dims)
        self.conv3 = GCNConv(hidden_dims, num_outputs)
        self.dropout = t.nn.Dropout(0.3)
        self.residual = t.nn.Linear(num_features, num_outputs)
        self.res_weight = t.nn.Parameter(t.tensor(2.0))

    def forward(self, x, edge_index):
        residual = self.residual(x) * self.res_weight
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x + residual

# Define L1 loss function directly
def l1_loss(x, y):
    return t.mean(t.abs(x - y))

# Pre-compute t2m_f statistics for denormalization
t2m_f_mean = 42.36  # °F
t2m_f_std = 21.75   # °F

# Define cities and their node indices
cities_nodes = [
    ("Los Angeles", 15195),
    ("San Francisco", 11623),
    ("Seattle", 2381),
    ("Missoula", 2888),
    ("Salt Lake City", 8821),
    ("Denver", 9797),
    ("Phoenix", 15694),
    ("San Antonio", 19540),
    ("New Orleans", 19100),
    ("Kansas City, MO", 10570),
    ("Chicago", 7733),
    ("New York City", 8973),
    ("Boston", 7563),
    ("Philadelphia", 9679),
    ("Atlanta", 15567),
    ("Miami", 23188),
]

# Testing function with iterative predictions
def test_model_iterative(model_path, inputs_tensor, targets_tensor, edge_index, num_nodes, device, t2m_f_mean, t2m_f_std, lat_subset, lon_subset):
    model = WeatherGNN(num_features=15, hidden_dims=128, num_outputs=1).to(device)
    model.load_state_dict(t.load(model_path))
    model.eval()
    print(f"Model loaded from {model_path}")

    # Validate inputs
    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Targets tensor shape: {targets_tensor.shape}")
    if inputs_tensor.shape[0] < 2 or targets_tensor.shape[0] < 2:
        raise ValueError("Input and target tensors must have at least 2 time steps for prediction.")

    # Iterative prediction
    test_preds = []
    current_input = inputs_tensor[0].clone()  # Shape: [num_nodes, num_features, 1]
    t2m_f_idx = 0  # Index of t2m_f in the feature dimension

    for t_step in range(inputs_tensor.shape[0] - 1):
        input_x = current_input.reshape(num_nodes, -1)  # Shape: [num_nodes, num_features]
        with t.no_grad():
            out = model(input_x, edge_index)  # Shape: [num_nodes, 1]
        test_preds.append(out)
        if t_step < inputs_tensor.shape[0] - 2:
            current_input = inputs_tensor[t_step + 1].clone()
            normalized_pred = (out - t2m_f_mean) / t2m_f_std
            current_input[:, t2m_f_idx, 0] = normalized_pred.squeeze(-1)

    test_preds = t.stack(test_preds)  # Shape: [num_time_steps-1, num_nodes, 1]
    test_trues = targets_tensor[:-1, :, 5, :].reshape(targets_tensor.shape[0]-1, num_nodes, 1)

    # Denormalize for evaluation and plotting
    preds_t2m = test_preds * t2m_f_std + t2m_f_mean
    trues_t2m = test_trues * t2m_f_std + t2m_f_mean
    mae_t2m = t.mean(t.abs(preds_t2m - trues_t2m)).item()
    rmse_t2m = t.sqrt(t.mean((preds_t2m - trues_t2m) ** 2)).item()
    print(f"Iterative t2m_f L1 norm (°F): {mae_t2m:.2f}")
    print(f"Iterative t2m_f RMSE (°F): {rmse_t2m:.2f}")

    # Move to CPU for plotting
    preds_t2m = preds_t2m.cpu()
    trues_t2m = trues_t2m.cpu()

    # Plot variation at specific cities in subplots
    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    axes = axes.flatten()
    for idx, (city, node) in enumerate(cities_nodes):
        ax = axes[idx]
        ax.plot(preds_t2m[:, node, 0].numpy(), label='Iterative Pred', linestyle='--', color='red')
        ax.plot(trues_t2m[:, node, 0].numpy(), label='True', linestyle='-', color='green')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Temperature (°F)')
        ax.set_title(f'{city}')
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    plt.savefig('t2m_f_iterative_variation_cities_subplots.png')
    plt.show()

    # Debug and plot spatial difference for first time step
    first_pred = preds_t2m[0, :, 0].numpy()
    first_true = trues_t2m[0, :, 0].numpy()
    print(f"First predicted t2m_f shape: {first_pred.shape}")
    print(f"First true t2m_f shape: {first_true.shape}")
    print(f"First predicted t2m_f sample: {first_pred[:5]}")
    print(f"First true t2m_f sample: {first_true[:5]}")

    # Check for NaN or infinite values
    if np.any(np.isnan(first_pred)) or np.any(np.isnan(first_true)):
        raise ValueError("NaN values detected in predictions or true values.")
    if np.any(np.isinf(first_pred)) or np.any(np.isinf(first_true)):
        raise ValueError("Infinite values detected in predictions or true values.")

    diff = np.abs(first_pred - first_true)
    print(f"Difference min: {np.min(diff):.2f}, max: {np.max(diff):.2f}, mean: {np.mean(diff):.2f}")

    # Reshape difference to 2D grid
    if diff.shape[0] != num_nodes:
        raise ValueError(f"Expected {num_nodes} nodes, but got {diff.shape[0]}")
    if len(lat_subset) * len(lon_subset) != num_nodes:
        raise ValueError(f"Grid size mismatch: lat_subset ({len(lat_subset)}) * lon_subset ({len(lon_subset)}) != num_nodes ({num_nodes})")
    diff_grid = diff.reshape(len(lat_subset), len(lon_subset))

    plt.figure(figsize=(15, 5))
    contour = plt.contourf(lon_subset, lat_subset, diff_grid, cmap='Blues', levels=np.linspace(0, max(np.max(diff), 1), 21), alpha=0.8)
    plt.colorbar(label='Absolute Difference (°F)')
    # Overlay darker latitude and longitude grid lines
    plt.grid(True, linestyle='--', color='black', alpha=0.7)
    plt.xticks(ticks=np.linspace(min(lon_subset), max(lon_subset), 10), rotation=45)
    plt.yticks(ticks=np.linspace(min(lat_subset), max(lat_subset), 10))
    plt.title('Iterative Absolute Difference (Pred - True) for t2m_f at First Test Time Step')
    plt.xlabel('Longitude (°E)')
    plt.ylabel('Latitude (°N)')
    save_path = 't2m_f_iterative_grid_difference.png'
    plt.savefig(save_path)
    print(f"Spatial grid plot saved to: {os.path.abspath(save_path)}")
    plt.show()

# Run the test with iterative predictions
test_model_iterative(
    model_path=r'F:\weather_forecasting\notebooks\final project\models\Final Model\final_model_ext_training.pth',
    inputs_tensor=temp_test_inputs,
    targets_tensor=temp_test_targets,
    edge_index=edge_index,
    num_nodes=num_nodes,
    device=device,
    t2m_f_mean=t2m_f_mean,
    t2m_f_std=t2m_f_std,
    lat_subset=lat_subset,
    lon_subset=lon_subset
)

# Testing all 8 models using iterative approach

In [ ]:
import time
import numpy as np
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Ensure plots display inline in Jupyter Notebook
%matplotlib inline

device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Directories
model_dir = r'F:\weather_forecasting\notebooks\final project\models\paths'
png_dir = r'F:\weather_forecasting\notebooks\final project\png'

# Load tensors
train_inputs_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/train_inputs_norm.pt').to(device)
train_targets_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/train_targets_norm.pt').to(device)
print(f"Tensors loaded: train inputs {train_inputs_tensor.shape}, train targets {train_targets_tensor.shape}")
print(f"Normalized t2m_f mean: {train_targets_tensor[:, :, 5, :].mean().item():.2f}, std: {train_targets_tensor[:, :, 5, :].std().item():.2f}")

# Temporary Train/Test split
train_size = int(0.9 * train_inputs_tensor.shape[0])
print(f"Train size: {train_size}")
temp_train_inputs = train_inputs_tensor[:train_size]
temp_train_targets = train_targets_tensor[:train_size]
temp_test_inputs = train_inputs_tensor[train_size:]
temp_test_targets = train_targets_tensor[train_size:]
print(f"Train/test split: train shape {temp_train_inputs.shape}, test shape {temp_test_inputs.shape}")

# Move tensors to GPU upfront
temp_train_inputs = temp_train_inputs.to(device)
temp_train_targets = temp_train_targets.to(device)
temp_test_inputs = temp_test_inputs.to(device)
temp_test_targets = temp_test_targets.to(device)

# Define grid and edge index
num_nodes = 23937
k = 8
lat_subset = np.linspace(50, 25, 101)
lon_subset = np.linspace(235, 294, 237)
coords = np.stack(np.meshgrid(lat_subset, lon_subset, indexing='ij'), axis=-1).reshape(-1, 2)
nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)
_, indices = nbrs.kneighbors(coords)
edge_index = t.tensor(np.stack([np.repeat(np.arange(num_nodes), k), indices[:, 1:].flatten()]), dtype=t.long).to(device)

# Model definition
class WeatherGNN(t.nn.Module):
    def __init__(self, num_features=15, hidden_dims=128, num_outputs=1):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dims)
        self.conv2 = GCNConv(hidden_dims, hidden_dims)
        self.conv3 = GCNConv(hidden_dims, num_outputs)
        self.dropout = t.nn.Dropout(0.3)
        self.residual = t.nn.Linear(num_features, num_outputs)
        self.res_weight = t.nn.Parameter(t.tensor(2.0))

    def forward(self, x, edge_index):
        residual = self.residual(x) * self.res_weight
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x + residual

# Define L1 loss function
def l1_loss(x, y):
    return t.mean(t.abs(x - y))

# Pre-compute t2m_f statistics for denormalization
t2m_f_mean = 42.36  # °F
t2m_f_std = 21.75   # °F

# Compute node indices for 16 major U.S. cities
cities = [
    ("Los Angeles", 34.05, -118.24),
    ("San Francisco", 37.77, -122.42),
    ("Seattle", 47.61, -122.33),
    ("Missoula", 46.87, -113.99),
    ("Salt Lake City", 40.76, -111.89),
    ("Denver", 39.74, -104.99),
    ("Phoenix", 33.45, -112.07),
    ("San Antonio", 29.42, -98.49),
    ("New Orleans", 29.95, -90.07),
    ("Kansas City", 39.10, -94.58),
    ("Chicago", 41.88, -87.63),
    ("New York City", 40.71, -74.01),
    ("Boston", 42.36, -71.06),
    ("Philadelphia", 39.95, -75.17),
    ("Atlanta", 33.75, -84.39),
    ("Miami", 25.76, -80.19)
]

nodes_to_plot = []
node_labels = []
for city, lat, lon in cities:
    # Convert longitude to 0-360 range
    lon = lon if lon > 0 else 360 + lon
    # Find nearest latitude and longitude indices
    lat_idx = np.argmin(np.abs(lat_subset - lat))
    lon_idx = np.argmin(np.abs(lon_subset - lon))
    # Compute node index
    node_idx = (lat_idx * 237) + lon_idx
    nodes_to_plot.append(node_idx)
    node_labels.append(city)
    print(f"{city}: Lat {lat:.2f}°N, Lon {lon:.2f}°E -> Node {node_idx} (Lat idx: {lat_idx}, Lon idx: {lon_idx})")

# Testing function with iterative predictions
def test_model_iterative(model_path, inputs_tensor, targets_tensor, edge_index, num_nodes, device, t2m_f_mean, t2m_f_std, lat_subset, lon_subset, plot_suffix=''):
    model = WeatherGNN(num_features=15, num_outputs=1).to(device)
    model.load_state_dict(t.load(model_path))
    model.eval()
    print(f"Model loaded from {model_path}")

    # Validate inputs
    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Targets tensor shape: {targets_tensor.shape}")
    if inputs_tensor.shape[0] < 2 or targets_tensor.shape[0] < 2:
        raise ValueError("Input and target tensors must have at least 2 time steps for prediction.")

    # Iterative prediction
    test_preds = []
    current_input = inputs_tensor[0].clone()  # Shape: [num_nodes, num_features, 1]
    t2m_f_idx = 0  # Index of t2m_f in the feature dimension

    for t_step in range(inputs_tensor.shape[0] - 1):
        input_x = current_input.reshape(num_nodes, -1)  # Shape: [num_nodes, num_features]
        with t.no_grad():
            out = model(input_x, edge_index)  # Shape: [num_nodes, 1]
        test_preds.append(out)

        if t_step < inputs_tensor.shape[0] - 2:
            current_input = inputs_tensor[t_step + 1].clone()
            normalized_pred = (out - t2m_f_mean) / t2m_f_std
            current_input[:, t2m_f_idx, 0] = normalized_pred.squeeze(-1)

    test_preds = t.stack(test_preds)
    test_trues = targets_tensor[:-1, :, 5, :].reshape(targets_tensor.shape[0]-1, num_nodes, 1)

    preds_t2m = test_preds * t2m_f_std + t2m_f_mean
    trues_t2m = test_trues * t2m_f_std + t2m_f_mean
    mae_t2m = t.mean(t.abs(preds_t2m - trues_t2m)).item()
    rmse_t2m = t.sqrt(t.mean((preds_t2m - trues_t2m) ** 2)).item()
    print(f"Iterative t2m_f L1 norm (°F): {mae_t2m:.2f}")
    print(f"Iterative t2m_f RMSE (°F): {rmse_t2m:.2f}")

    preds_t2m = preds_t2m.cpu()
    trues_t2m = trues_t2m.cpu()

    # Plot variation at specific cities in subplots
    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    axes = axes.flatten()
    for idx, node, city in zip(range(len(nodes_to_plot)), nodes_to_plot, node_labels):
        ax = axes[idx]
        ax.plot(preds_t2m[:, node, 0].numpy(), label='Iterative Pred', linestyle='--', color='red')
        ax.plot(trues_t2m[:, node, 0].numpy(), label='True', linestyle='-', color='green')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Temperature (°F)')
        ax.set_title(f'Predicted vs True t2m_f at {city}')
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    save_path = os.path.join(png_dir, f't2m_f_iterative_variation_cities_subplots_{plot_suffix}.png')
    plt.savefig(save_path)
    plt.show()

    # Plot spatial difference with U.S. outline
    first_pred = preds_t2m[0, :, 0].numpy()
    first_true = trues_t2m[0, :, 0].numpy()
    diff = np.abs(first_pred - first_true)
    diff_grid = diff.reshape(len(lat_subset), len(lon_subset))

    fig, ax = plt.subplots(figsize=(15, 5), subplot_kw={'projection': ccrs.PlateCarree()})
    contour = ax.contourf(lon_subset, lat_subset, diff_grid, cmap='Pastel1', levels=np.linspace(0, max(np.max(diff), 1), 21), transform=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='black', linewidth=0.5)
    plt.colorbar(contour, label='Absolute Difference (°F)')
    ax.set_title('Iterative Absolute Difference (Pred - True) for t2m_f at First Test Time Step')
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    save_path = os.path.join(png_dir, f't2m_f_iterative_grid_difference_{plot_suffix}.png')
    plt.savefig(save_path)
    plt.show()

# Model files
model_files = [
    'model_lr0p001_adaptive_20250418_083926.pth',
    'model_lr0p001_stationary_20250418_083926.pth',
    'model_lr0p01_adaptive_20250418_083926.pth',
    'model_lr0p01_stationary_20250418_083926.pth',
    'model_lr0p0005_adaptive_20250418_083926.pth',
    'model_lr0p0005_stationary_20250418_083926.pth',
    'model_lr0p005_adaptive_20250418_083926.pth',
    'model_lr0p005_stationary_20250418_083926.pth'
]

# Loop over each model
for model_file in model_files:
    print(f"\nRunning iterative predictions for model: {model_file}")
    model_id = os.path.basename(model_file).replace('.pth', '')
    test_model_iterative(
        model_path=os.path.join(model_dir, model_file),
        inputs_tensor=temp_test_inputs,
        targets_tensor=temp_test_targets,
        edge_index=edge_index,
        num_nodes=num_nodes,
        device=device,
        t2m_f_mean=t2m_f_mean,
        t2m_f_std=t2m_f_std,
        lat_subset=lat_subset,
        lon_subset=lon_subset,
        plot_suffix=model_id
    )

# Final Evaluation

## Processing April Data

In [ ]:
import time
import xarray as xr
import numpy as np
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
import glob
import os

# Ensure plots display inline in Jupyter Notebook
%matplotlib inline

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# File paths
file_path_test_a = "../nc/accum_test.nc"
file_path_test_i = "../nc/instant_test.nc"
file_path_test_p = "../nc/pressure_test.nc"

# Open datasets with larger chunks
# Testing Data [April 1, 2025 - April 7, 2025]
start = time.time()
ds_a_test = xr.open_dataset(file_path_test_a, chunks={'valid_time': 100})
ds_i_test = xr.open_dataset(file_path_test_i, chunks={'valid_time': 100})
ds_p_test = xr.open_dataset(file_path_test_p, chunks={'valid_time':100})
# Drop expver if present
if 'expver' in ds_a_test.coords:
    ds_a_train = ds_a_test.drop_vars('expver')
if 'expver' in ds_i_test.coords:
    ds_i_train = ds_i_test.drop_vars('expver')
if 'expver' in ds_p_test.coords:
    ds_p_train = ds_p_test.drop_vars('expver')
print(f"Testing datasets opened in {time.time() - start:.2f}s: ds_i_test shape {ds_i_test.dims}, ds_a_test shape {ds_a_test.dims}, ds_p_test shape {ds_p_test.dims}")

# Define variable names
i_vars = ['t2m', 'd2m', 'tcc', 'sp', 'u10', 'v10', 'stl1', 'blh']
a_vars = ['tp', 'sshf', 'slhf', 'ssrd', 'strd']
p_vars = ['t', 'q']
print(f"Variables defined: instant {i_vars}, accum {a_vars}, pressure {p_vars}")

# Merge datasets
# Testing Data
start = time.time()
ds_test_merge = xr.merge([ds_i_test[i_vars], ds_a_test[a_vars], ds_p_test[p_vars]])
print(f"Testing datasets merged in {time.time() - start:.2f}s: shape {ds_test_merge.dims}, dtype {ds_test_merge[i_vars[0]].dtype}")

# Convert to Fahrenheit
def kelvin_to_fahrenheit(temp_k):
    temp_f = (temp_k - 273.15) * 9/5 + 32
    return temp_f

# Converting to fahrenheit & adding temperature spread
start = time.time()
ds_test_merge = ds_test_merge.assign(t2m_f=lambda ds: kelvin_to_fahrenheit(ds['t2m']))
ds_test_merge = ds_test_merge.assign(d2m_f=lambda ds: kelvin_to_fahrenheit(ds['d2m']))
ds_test_merge = ds_test_merge.assign(t_f=lambda ds: kelvin_to_fahrenheit(ds['t']))
ds_test_merge = ds_test_merge.assign(t_spread_f=lambda ds: ds['t2m_f']-ds['d2m_f'])
print(f"Converting to F in {time.time() - start:.2f}")

# Dropping the original temperature variables as well as d2m_f
ds_test_merge = ds_test_merge.drop_vars(['t2m', 'd2m', 'd2m_f', 't'])

# Update variables
i_vars = ['t2m_f', 't_spread_f', 'tcc', 'sp', 'u10', 'v10', 'stl1', 'blh']
a_vars = ['tp', 'sshf', 'slhf', 'ssrd', 'strd']
p_vars = ['t_f', 'q']
all_vars = a_vars+i_vars+p_vars

# Save intermediates
start = time.time()
ds_test_merge.to_netcdf('f:/weather_forecasting/ds_train.nc')
print(f"Saved intermediates in {time.time() - start:.2f}s: test shape {ds_test_merge.dims}")

# Stack spatial dimensions
start = time.time()
ds_test_stacked = ds_test_merge[all_vars].stack(node=('latitude', 'longitude'))
print(f"Stacked spatial dims in {time.time() - start:.2f}s: test shape {ds_test_stacked.dims}")

# Create inputs and targets (all time steps)
start = time.time()
# Testing
total_test_time_steps = ds_test_stacked.dims['valid_time']
print(f"Total testing time steps: {total_test_time_steps}")
test_inputs = ds_test_stacked.isel(valid_time=slice(0, total_test_time_steps - 1))
test_targets = ds_test_stacked.isel(valid_time=slice(1, total_test_time_steps))

# Tensor build
# Convert to numpy array
start = time.time()
test_inputs_array = test_inputs.to_array().values.transpose(1, 2, 0, 3)
test_targets_array = test_targets.to_array().values.transpose(1, 2, 0, 3)
print(f"Converted to arrays in {time.time() - start:.2f}s: test inputs shape {test_inputs_array.shape}, test targets shape {test_targets_array.shape}, dtype {test_inputs_array.dtype}")

# Convert to pytorch tensors
start = time.time()
test_inputs_tensor = t.tensor(test_inputs_array, dtype=t.float32).to(device)
test_targets_tensor = t.tensor(test_targets_array, dtype=t.float32).to(device)
print(f"Tensors built in {time.time() - start:.2f}s: test inputs shape {test_inputs_tensor.shape}, test targets shape {test_targets_tensor.shape}, device {test_inputs_tensor.device}")

# Normalize tensors (using mean and std from the testing dataset only)
start = time.time()
test_inputs_mean = test_inputs_tensor.mean(dim=(0, 1), keepdim=True)
test_inputs_std = test_inputs_tensor.std(dim=(0, 1), keepdim=True)
test_targets_mean = test_targets_tensor.mean(dim=(0, 1), keepdim=True)
test_targets_std = test_targets_tensor.std(dim=(0, 1), keepdim=True)

# Normalize testing tensors
test_inputs_tensor = (test_inputs_tensor - test_inputs_mean) / (test_inputs_std + 1e-8)
test_targets_tensor = (test_targets_tensor - test_targets_mean) / (test_targets_std + 1e-8)

# Save normalized tensors
start = time.time()
t.save(test_inputs_tensor.cpu(), 'f:/weather_forecasting/notebooks/final project/tensors/test_inputs_norm.pt')
t.save(test_targets_tensor.cpu(), 'f:/weather_forecasting/notebooks/final project/tensors/test_targets_norm.pt')
print(f"Normalized tensors saved in {time.time() - start:.2f}s to f:/weather_forecasting/notebooks/final project/tensors")

## Testing Against Persistence Model Using Processed April Data

In [ ]:
import time
import numpy as np
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
import glob
import os

# Ensure plots display inline in Jupyter Notebook
%matplotlib inline

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

test_inputs_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/test_inputs_norm.pt').to(device)
test_targets_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/test_targets_norm.pt').to(device)
print(f"Tensors loaded: test inputs {test_inputs_tensor.shape}, test targets {test_targets_tensor.shape}")
print(f"Normalized t2m_f mean: {test_targets_tensor[:, :, 5, :].mean().item():.2f}, std: {test_targets_tensor[:, :, 5, :].std().item():.2f}")

# Define grid and edge index (must match training setup)
num_nodes = 23937
k = 8
lat_subset = np.linspace(50, 25, 101)
lon_subset = np.linspace(235, 294, 237)
coords = np.stack(np.meshgrid(lat_subset, lon_subset, indexing='ij'), axis=-1).reshape(-1, 2)
nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)
_, indices = nbrs.kneighbors(coords)
edge_index = t.tensor(np.stack([np.repeat(np.arange(num_nodes), k), indices[:, 1:].flatten()]), dtype=t.long).to(device)

# Model definition (must match the trained model)
class WeatherGNN(t.nn.Module):
    def __init__(self, num_features=15, hidden_dims=128, num_outputs=1):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dims)
        self.conv2 = GCNConv(hidden_dims, hidden_dims)
        self.conv3 = GCNConv(hidden_dims, num_outputs)
        self.dropout = t.nn.Dropout(0.3)
        self.residual = t.nn.Linear(num_features, num_outputs)
        self.res_weight = t.nn.Parameter(t.tensor(2.0))

    def forward(self, x, edge_index):
        residual = self.residual(x) * self.res_weight
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x + residual

# Define L1 loss function directly
def l1_loss(x, y):
    return t.mean(t.abs(x - y))

# Pre-compute t2m_f statistics for denormalization (must match training)
t2m_f_mean = 42.36  # °F
t2m_f_std = 21.75   # °F

# Define cities and their node indices
cities_nodes = [
    ("Los Angeles", 15195),
    ("San Francisco", 11623),
    ("Seattle", 2381),
    ("Missoula", 2888),
    ("Salt Lake City", 8821),
    ("Denver", 9797),
    ("Phoenix", 15694),
    ("San Antonio", 19540),
    ("New Orleans", 19100),
    ("Kansas City, MO", 10570),
    ("Chicago", 7733),
    ("New York City", 8973),
    ("Boston", 7563),
    ("Philadelphia", 9679),
    ("Atlanta", 15567),
    ("Miami", 23188),
]

# Testing function with iterative predictions and corrected persistence model
def test_model_iterative(model_path, inputs_tensor, targets_tensor, edge_index, num_nodes, device, t2m_f_mean, t2m_f_std, lat_subset, lon_subset):
    model = WeatherGNN(num_features=15, hidden_dims=128, num_outputs=1).to(device)
    model.load_state_dict(t.load(model_path))
    model.eval()
    print(f"Model loaded from {model_path}")

    # Validate inputs
    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Targets tensor shape: {targets_tensor.shape}")
    if inputs_tensor.shape[0] < 2 or targets_tensor.shape[0] < 2:
        raise ValueError("Input and target tensors must have at least 2 time steps for prediction.")

    # Iterative prediction for GNN model
    test_preds = []
    current_input = inputs_tensor[0].clone()  # Shape: [num_nodes, num_features, 1]
    t2m_f_idx = 0  # Index of t2m_f in the feature dimension

    # Persistence model: predict all future steps as the value at time 0
    test_persist_preds = []
    persist_pred = targets_tensor[0, :, 5, :].reshape(1, num_nodes, 1)  # Value at t=0, Shape: [1, num_nodes, 1]

    for t_step in range(inputs_tensor.shape[0] - 1):
        input_x = current_input.reshape(num_nodes, -1)  # Shape: [num_nodes, num_features]
        with t.no_grad():
            out = model(input_x, edge_index)  # Shape: [num_nodes, 1]
        test_preds.append(out)
        test_persist_preds.append(persist_pred.squeeze(0))  # Use t=0 value for all predictions
        if t_step < inputs_tensor.shape[0] - 2:
            current_input = inputs_tensor[t_step + 1].clone()
            normalized_pred = (out - t2m_f_mean) / t2m_f_std
            current_input[:, t2m_f_idx, 0] = normalized_pred.squeeze(-1)

    test_preds = t.stack(test_preds)  # Shape: [num_time_steps-1, num_nodes, 1]
    test_persist_preds = t.stack(test_persist_preds)  # Shape: [num_time_steps-1, num_nodes, 1]
    test_trues = targets_tensor[1:, :, 5, :].reshape(targets_tensor.shape[0]-1, num_nodes, 1)  # True values for t=1 to T-1

    # Denormalize for evaluation and plotting
    preds_t2m = test_preds * t2m_f_std + t2m_f_mean
    persist_t2m = test_persist_preds * t2m_f_std + t2m_f_mean
    trues_t2m = test_trues * t2m_f_std + t2m_f_mean
    mae_t2m = t.mean(t.abs(preds_t2m - trues_t2m)).item()
    rmse_t2m = t.sqrt(t.mean((preds_t2m - trues_t2m) ** 2)).item()
    persist_mae_t2m = t.mean(t.abs(persist_t2m - trues_t2m)).item()
    persist_rmse_t2m = t.sqrt(t.mean((persist_t2m - trues_t2m) ** 2)).item()
    print(f"GNN Iterative t2m_f L1 norm (°F): {mae_t2m:.2f}")
    print(f"GNN Iterative t2m_f RMSE (°F): {rmse_t2m:.2f}")
    print(f"Persistence t2m_f L1 norm (°F): {persist_mae_t2m:.2f}")
    print(f"Persistence t2m_f RMSE (°F): {persist_rmse_t2m:.2f}")

    # Move to CPU for plotting
    preds_t2m = preds_t2m.cpu()
    persist_t2m = persist_t2m.cpu()
    trues_t2m = trues_t2m.cpu()

    # Plot variation at specific cities in subplots with persistence
    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    axes = axes.flatten()
    for idx, (city, node) in enumerate(cities_nodes):
        ax = axes[idx]
        ax.plot(preds_t2m[:, node, 0].numpy(), label='GNN Pred', linestyle='--', color='red')
        ax.plot(persist_t2m[:, node, 0].numpy(), label='Persistence Pred', linestyle=':', color='blue')
        ax.plot(trues_t2m[:, node, 0].numpy(), label='True', linestyle='-', color='green')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Temperature (°F)')
        ax.set_title(f'{city}')
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    plt.savefig('t2m_f_iterative_variation_cities_subplots_with_persistence.png')
    plt.show()

    # Debug and plot spatial difference for first time step
    first_pred = preds_t2m[0, :, 0].numpy()
    first_true = trues_t2m[0, :, 0].numpy()
    print(f"First predicted t2m_f shape: {first_pred.shape}")
    print(f"First true t2m_f shape: {first_true.shape}")
    print(f"First predicted t2m_f sample: {first_pred[:5]}")
    print(f"First true t2m_f sample: {first_true[:5]}")

    diff = np.abs(first_pred - first_true)
    print(f"Difference min: {np.min(diff):.2f}, max: {np.max(diff):.2f}, mean: {np.mean(diff):.2f}")

    # Reshape difference to 2D grid
    if diff.shape[0] != num_nodes:
        raise ValueError(f"Expected {num_nodes} nodes, but got {diff.shape[0]}")
    if len(lat_subset) * len(lon_subset) != num_nodes:
        raise ValueError(f"Grid size mismatch: lat_subset ({len(lat_subset)}) * lon_subset ({len(lon_subset)}) != num_nodes ({num_nodes})")
    diff_grid = diff.reshape(len(lat_subset), len(lon_subset))

    plt.figure(figsize=(15, 5))
    contour = plt.contourf(lon_subset, lat_subset, diff_grid, cmap='Pastel1', levels=np.linspace(0, max(np.max(diff), 1), 21), alpha=0.8)
    plt.colorbar(label='Absolute Difference (°F)')
    # Overlay darker latitude and longitude grid lines
    plt.grid(True, linestyle='--', color='black', alpha=0.7)
    plt.xticks(ticks=np.linspace(min(lon_subset), max(lon_subset), 10), rotation=45)
    plt.yticks(ticks=np.linspace(min(lat_subset), max(lat_subset), 10))
    plt.title('GNN Iterative Absolute Difference (Pred - True) for t2m_f at First Test Time Step')
    plt.xlabel('Longitude (°E)')
    plt.ylabel('Latitude (°N)')
    save_path = 't2m_f_iterative_grid_difference.png'
    plt.savefig(save_path)
    print(f"Spatial grid plot saved to: {os.path.abspath(save_path)}")
    plt.show()

# Run the test with iterative predictions
test_model_iterative(
    model_path=r'f:\weather_forecasting\notebooks\final project\models\Final Model\final_model.pth',
    inputs_tensor=test_inputs_tensor,
    targets_tensor=test_targets_tensor,
    edge_index=edge_index,
    num_nodes=num_nodes,
    device=device,
    t2m_f_mean=t2m_f_mean,
    t2m_f_std=t2m_f_std,
    lat_subset=lat_subset,
    lon_subset=lon_subset
)

In [ ]:
import time
import numpy as np
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
import glob
import os

# Ensure plots display inline in Jupyter Notebook
%matplotlib inline

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

test_inputs_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/test_inputs_norm.pt').to(device)
test_targets_tensor = t.load('f:/weather_forecasting/notebooks/final project/tensors/test_targets_norm.pt').to(device)
print(f"Tensors loaded: test inputs {test_inputs_tensor.shape}, test targets {test_targets_tensor.shape}")
print(f"Normalized t2m_f mean: {test_targets_tensor[:, :, 5, :].mean().item():.2f}, std: {test_targets_tensor[:, :, 5, :].std().item():.2f}")

# Define grid and edge index (must match training setup)
num_nodes = 23937
k = 8
lat_subset = np.linspace(50, 25, 101)
lon_subset = np.linspace(235, 294, 237)
coords = np.stack(np.meshgrid(lat_subset, lon_subset, indexing='ij'), axis=-1).reshape(-1, 2)
nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)
_, indices = nbrs.kneighbors(coords)
edge_index = t.tensor(np.stack([np.repeat(np.arange(num_nodes), k), indices[:, 1:].flatten()]), dtype=t.long).to(device)

# Model definition (must match the trained model)
class WeatherGNN(t.nn.Module):
    def __init__(self, num_features=15, hidden_dims=128, num_outputs=1):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dims)
        self.conv2 = GCNConv(hidden_dims, hidden_dims)
        self.conv3 = GCNConv(hidden_dims, num_outputs)
        self.dropout = t.nn.Dropout(0.3)
        self.residual = t.nn.Linear(num_features, num_outputs)
        self.res_weight = t.nn.Parameter(t.tensor(2.0))

    def forward(self, x, edge_index):
        residual = self.residual(x) * self.res_weight
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x + residual

# Define L1 loss function directly
def l1_loss(x, y):
    return t.mean(t.abs(x - y))

# Pre-compute t2m_f statistics for denormalization (must match training)
t2m_f_mean = 42.36  # °F
t2m_f_std = 21.75   # °F

# Define cities and their node indices
cities_nodes = [
    ("Los Angeles", 15195),
    ("San Francisco", 11623),
    ("Seattle", 2381),
    ("Missoula", 2888),
    ("Salt Lake City", 8821),
    ("Denver", 9797),
    ("Phoenix", 15694),
    ("San Antonio", 19540),
    ("New Orleans", 19100),
    ("Kansas City, MO", 10570),
    ("Chicago", 7733),
    ("New York City", 8973),
    ("Boston", 7563),
    ("Philadelphia", 9679),
    ("Atlanta", 15567),
    ("Miami", 23188),
]

# Testing function with iterative predictions and corrected persistence model
def test_model_iterative(model_path, inputs_tensor, targets_tensor, edge_index, num_nodes, device, t2m_f_mean, t2m_f_std, lat_subset, lon_subset):
    model = WeatherGNN(num_features=15, hidden_dims=128, num_outputs=1).to(device)
    model.load_state_dict(t.load(model_path))
    model.eval()
    print(f"Model loaded from {model_path}")

    # Validate inputs
    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Targets tensor shape: {targets_tensor.shape}")
    if inputs_tensor.shape[0] < 2 or targets_tensor.shape[0] < 2:
        raise ValueError("Input and target tensors must have at least 2 time steps for prediction.")

    # Iterative prediction for GNN model
    test_preds = []
    current_input = inputs_tensor[0].clone()  # Shape: [num_nodes, num_features, 1]
    t2m_f_idx = 0  # Index of t2m_f in the feature dimension

    # Persistence model: predict all future steps as the value at time 0
    test_persist_preds = []
    persist_pred = targets_tensor[0, :, 5, :].reshape(1, num_nodes, 1)  # Value at t=0, Shape: [1, num_nodes, 1]

    for t_step in range(inputs_tensor.shape[0] - 1):
        input_x = current_input.reshape(num_nodes, -1)  # Shape: [num_nodes, num_features]
        with t.no_grad():
            out = model(input_x, edge_index)  # Shape: [num_nodes, 1]
        test_preds.append(out)
        test_persist_preds.append(persist_pred.squeeze(0))  # Use t=0 value for all predictions
        if t_step < inputs_tensor.shape[0] - 2:
            current_input = inputs_tensor[t_step + 1].clone()
            normalized_pred = (out - t2m_f_mean) / t2m_f_std
            current_input[:, t2m_f_idx, 0] = normalized_pred.squeeze(-1)

    test_preds = t.stack(test_preds)  # Shape: [num_time_steps-1, num_nodes, 1]
    test_persist_preds = t.stack(test_persist_preds)  # Shape: [num_time_steps-1, num_nodes, 1]
    test_trues = targets_tensor[1:, :, 5, :].reshape(targets_tensor.shape[0]-1, num_nodes, 1)  # True values for t=1 to T-1

    # Denormalize for evaluation and plotting
    preds_t2m = test_preds * t2m_f_std + t2m_f_mean
    persist_t2m = test_persist_preds * t2m_f_std + t2m_f_mean
    trues_t2m = test_trues * t2m_f_std + t2m_f_mean
    mae_t2m = t.mean(t.abs(preds_t2m - trues_t2m)).item()
    rmse_t2m = t.sqrt(t.mean((preds_t2m - trues_t2m) ** 2)).item()
    persist_mae_t2m = t.mean(t.abs(persist_t2m - trues_t2m)).item()
    persist_rmse_t2m = t.sqrt(t.mean((persist_t2m - trues_t2m) ** 2)).item()
    print(f"GNN Iterative t2m_f L1 norm (°F): {mae_t2m:.2f}")
    print(f"GNN Iterative t2m_f RMSE (°F): {rmse_t2m:.2f}")
    print(f"Persistence t2m_f L1 norm (°F): {persist_mae_t2m:.2f}")
    print(f"Persistence t2m_f RMSE (°F): {persist_rmse_t2m:.2f}")

    # Move to CPU for plotting
    preds_t2m = preds_t2m.cpu()
    persist_t2m = persist_t2m.cpu()
    trues_t2m = trues_t2m.cpu()

    # Plot variation at specific cities in subplots with persistence
    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    axes = axes.flatten()
    for idx, (city, node) in enumerate(cities_nodes):
        ax = axes[idx]
        ax.plot(preds_t2m[:, node, 0].numpy(), label='GNN Pred', linestyle='--', color='red')
        ax.plot(persist_t2m[:, node, 0].numpy(), label='Persistence Pred', linestyle=':', color='blue')
        ax.plot(trues_t2m[:, node, 0].numpy(), label='True', linestyle='-', color='green')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Temperature (°F)')
        ax.set_title(f'{city}')
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    plt.savefig('t2m_f_iterative_variation_cities_subplots_with_persistence.png')
    plt.show()

    # Debug and plot spatial difference for first time step
    first_pred = preds_t2m[0, :, 0].numpy()
    first_true = trues_t2m[0, :, 0].numpy()
    print(f"First predicted t2m_f shape: {first_pred.shape}")
    print(f"First true t2m_f shape: {first_true.shape}")
    print(f"First predicted t2m_f sample: {first_pred[:5]}")
    print(f"First true t2m_f sample: {first_true[:5]}")

    diff = np.abs(first_pred - first_true)
    print(f"Difference min: {np.min(diff):.2f}, max: {np.max(diff):.2f}, mean: {np.mean(diff):.2f}")

    # Reshape difference to 2D grid
    if diff.shape[0] != num_nodes:
        raise ValueError(f"Expected {num_nodes} nodes, but got {diff.shape[0]}")
    if len(lat_subset) * len(lon_subset) != num_nodes:
        raise ValueError(f"Grid size mismatch: lat_subset ({len(lat_subset)}) * lon_subset ({len(lon_subset)}) != num_nodes ({num_nodes})")
    diff_grid = diff.reshape(len(lat_subset), len(lon_subset))

    plt.figure(figsize=(15, 5))
    contour = plt.contourf(lon_subset, lat_subset, diff_grid, cmap='Pastel1', levels=np.linspace(0, max(np.max(diff), 1), 21), alpha=0.8)
    plt.colorbar(label='Absolute Difference (°F)')
    # Overlay darker latitude and longitude grid lines
    plt.grid(True, linestyle='--', color='black', alpha=0.7)
    plt.xticks(ticks=np.linspace(min(lon_subset), max(lon_subset), 10), rotation=45)
    plt.yticks(ticks=np.linspace(min(lat_subset), max(lat_subset), 10))
    plt.title('GNN Iterative Absolute Difference (Pred - True) for t2m_f at First Test Time Step')
    plt.xlabel('Longitude (°E)')
    plt.ylabel('Latitude (°N)')
    save_path = 't2m_f_iterative_grid_difference.png'
    plt.savefig(save_path)
    print(f"Spatial grid plot saved to: {os.path.abspath(save_path)}")
    plt.show()

# Run the test with iterative predictions
test_model_iterative(
    model_path=r'f:\weather_forecasting\notebooks\final project\models\Final Model\final_model_ext_training.pth',
    inputs_tensor=test_inputs_tensor,
    targets_tensor=test_targets_tensor,
    edge_index=edge_index,
    num_nodes=num_nodes,
    device=device,
    t2m_f_mean=t2m_f_mean,
    t2m_f_std=t2m_f_std,
    lat_subset=lat_subset,
    lon_subset=lon_subset
)